In [2]:
# Install GDAL and Geopandas
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!pip install git+git://github.com/geopandas/geopandas.git --quiet
!pip install descartes --quiet
!pip install geopy
!pip install plotly_express
!pip install ipython-autotime

!pip install tqdm==4.36.0


Reading package lists...
Building dependency tree...
Reading state information...
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources
Suggested packages:
  python3-setuptools
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources python3-rtree
0 upgraded, 5 newly installed, 0 to remove and 11 not upgraded.
Need to get 671 kB of archives.
After this operation, 3,948 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2

In [3]:
!pip install tqdm==4.36.0

In [4]:
%load_ext autotime
import pandas as pd

import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

In [6]:
from google.colab import files

uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  

Saving All.plt to All.plt
User uploaded file "All.plt" with length 488569 bytes
time: 30.5 s


In [7]:
import io
df = pd.read_csv(io.BytesIO(uploaded[fn]))
df.columns = ['lat','lgt','allzero','alt','timestamp','date','time']
df.head()
df.shape

(7613, 7)

time: 33.5 ms


In [8]:

import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from sklearn.cluster import DBSCAN
from sklearn import metrics
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
%matplotlib inline

time: 646 ms


In [9]:
# define the number of kilometers in one radian
kms_per_radian = 637100.88

time: 748 µs


In [10]:
import io
df = pd.read_csv(io.BytesIO(uploaded[fn]))
df.columns = ['lat','lgt','allzero','alt','timestamp','date','time']
df.head()
df.shape

(7613, 7)

time: 22.5 ms


In [11]:

# represent points consistently as (lat, lon)
coords = df[["lat", "lgt"]].values

# define epsilon as 1.5 kilometers, converted to radians for use by haversine
epsilon = 1.5 / kms_per_radian

time: 8.7 ms


In [12]:
start_time = time.time()
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_

# get the number of clusters
num_clusters = len(set(cluster_labels))


# all done, print the outcome
message = 'Clustered {:,} points down to {:,} clusters, for {:.1f}% compression in {:,.2f} seconds'
print(message.format(len(df), num_clusters, 100*(1 - float(num_clusters) / len(df)), time.time()-start_time))
print('Silhouette coefficient: {:0.03f}'.format(metrics.silhouette_score(coords, cluster_labels)))


Clustered 7,613 points down to 663 clusters, for 91.3% compression in 0.60 seconds
Silhouette coefficient: -0.488
time: 1.76 s


In [13]:
# turn the clusters in to a pandas series, where each element is a cluster of points
clusters = pd.Series([coords[cluster_labels==n] for n in range(num_clusters)])

time: 22.8 ms


In [14]:
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

centermost_points = clusters.map(get_centermost_point)


time: 293 ms


In [15]:
# unzip the list of centermost points (lat, lon) tuples into separate lat and lon lists
lats, lons = zip(*centermost_points)

# from these lats/lons create a new df of one representative point for each cluster
rep_points = pd.DataFrame({'lgt':lons, 'lat':lats})
rep_points.tail()

,lgt,lat
658,116.318813,40.004252
659,116.318750,40.004387
660,116.318927,40.005666
661,116.319031,40.006118
662,116.319428,40.006571


time: 18.3 ms


In [16]:
# pull row from original data set where lat/lon match the lat/lon of each row of representative points
# that way we get the full details like city, country, and date from the original dataframe
rs = rep_points.apply(lambda row: df[(df['lat']==row['lat']) & (df['lgt']==row['lgt'])].iloc[0], axis=1)
#rs.to_csv('data/summer-travel-gps-dbscan.csv', encoding='utf-8')
rs.tail()

,lat,lgt,allzero,alt,timestamp,date,time
658,40.004252,116.318813,0,567,39763.106817,2008-11-11,02:33:49
659,40.004387,116.318750,0,546,39763.106875,2008-11-11,02:33:54
660,40.005666,116.318927,0,346,39763.107801,2008-11-11,02:35:14
661,40.006118,116.319031,0,313,39763.107975,2008-11-11,02:35:29
662,40.006571,116.319428,0,309,39763.108148,2008-11-11,02:35:44


time: 855 ms


In [17]:
"""from google.colab import files
rs.to_csv('filename.txt')
files.download('filename.txt')
"""

"from google.colab import files\nrs.to_csv('filename.txt')\nfiles.download('filename.txt')\n"

time: 2.3 ms


In [18]:
import plotly.graph_objects as go
import pandas as pd
from plotly.validators.scatter.marker import SymbolValidator

mapbox_access_token ="pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w"
raw_symbols = SymbolValidator().values
symbols = []
fig = go.Figure(go.Scattermapbox(
        lat=rs['lat'],
        lon=rs['lgt'],
        mode='markers',
        marker=go.scattermapbox.Marker(
        size = 25,
        color='rgb(172, 173, 242)',
        symbol= 'square-stroked'
        ),
        text=rs['date'],
    ))

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=rs['lat'][0],
            lon=rs['lgt'][0]
        ),
        pitch=0,
        zoom=16,
        style='light'
    ),
)

fig.show()


time: 268 ms


In [19]:
import plotly_express as px
import plotly
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")
px.scatter_mapbox(df, lat="lat", lon="lgt",  zoom=15)

time: 330 ms


In [20]:
import plotly_express as px
import plotly
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")
px.scatter_mapbox(rs, lat="lat", lon="lgt", zoom=15)

time: 322 ms


In [21]:
''' For Testing '''
import plotly.graph_objects as go
import pandas as pd
from plotly.validators.scatter.marker import SymbolValidator

fig = go.Figure()
mapbox_access_token ="pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w"
raw_symbols = SymbolValidator().values
symbols = []
fig.add_trace(go.Scattermapbox(
        lat=df['lat'],
        lon=df['lgt'],
        mode='markers',
        marker=go.scattermapbox.Marker(
        size = 10,
        color='rgb(172, 173, 242)'
        ),
        text=df['date'],
    ))

fig.add_trace(go.Scattermapbox(
        lat=rs['lat'],
        lon=rs['lgt'],
        mode='markers',
        marker=go.scattermapbox.Marker(
        size = 50,
        color='rgb(172, 173, 242)',
        symbol= 'square-stroked'
        ),
        text=rs['date'],
    ))

fig.add_trace(
    go.Scattermapbox(
    mode = "lines", fill = "toself",
    lon = [116.3184, 116.3184, 116.3194,116.3194, 116.3184, None, 116.319427, 116.319427, 116.320527, 116.320527, 116.319427, None, 116.320527, 116.320527, 116.321527,116.321527,116.320527, None, 100, 100, 80, 80, 100],
    lat = [39.984702, 39.985702, 39.985710, 39.984702, 39.984702, None, 39.984702, 39.985702,39.985702, 39.984702, 39.984702, None, 39.984702, 39.985702, 39.985710, 39.984702, 39.984702, None, 40, 50, 50, 40, 40])
    )

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=rs['lat'][0],
            lon=rs['lgt'][0]
        ),
        pitch=0,
        zoom=13,
        style='light'
    ),
)
fig.show()

time: 353 ms


In [22]:
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

import io
dflongtitude = pd.read_csv(io.BytesIO(uploaded[fn]))
dflongtitude.columns = ['A','B','C','D','E','F']
dflongtitude.head()





Saving 20081023025304.plt to 20081023025304.plt
User uploaded file "20081023025304.plt" with length 58568 bytes


ValueError: ignored

time: 1min 13s


In [ ]:
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
import io
dflatiude = pd.read_csv(io.BytesIO(uploaded[fn]))
dflatiude.columns = ['A','B','C','D','E','F']
dflatiude.head()


In [40]:
''' For Testing '''
import plotly.graph_objects as go
import pandas as pd
from plotly.validators.scatter.marker import SymbolValidator

fig = go.Figure()
mapbox_access_token ="pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w"
raw_symbols = SymbolValidator().values
symbols = []
fig.add_trace(go.Scattermapbox(
        lat=df['lat'],
        lon=df['lgt'],
        mode='markers',
        marker=go.scattermapbox.Marker(
        size = 10,
        color='rgb(172, 173, 242)'
        ),
        text=df['date'],
    ))


fig.add_trace(
    go.Scattermapbox(
    mode = "lines", fill = "none",
    #lon = [dflongtitude['A'],dflongtitude['B'],dflongtitude['C'],dflongtitude['D'],dflongtitude['E'],dflongtitude['F']],
    lon = [
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047,None,
           116.275446,116.275446,116.276446,116.276446,116.275446,None,116.276446,116.276446,116.27744600000001,116.27744600000001,116.276446,None,116.27744600000001,116.27744600000001,116.27844600000002,116.27844600000002,116.27744600000001,None,116.27844600000002,116.27844600000002,116.27944600000002,116.27944600000002,116.27844600000002,None,116.27944600000002,116.27944600000002,116.28044600000003,116.28044600000003,116.27944600000002,None,116.28044600000003,116.28044600000003,116.28144600000003,116.28144600000003,116.28044600000003,None,116.28144600000003,116.28144600000003,116.28244600000004,116.28244600000004,116.28144600000003,None,116.28244600000004,116.28244600000004,116.28344600000004,116.28344600000004,116.28244600000004,None,116.28344600000004,116.28344600000004,116.28444600000005,116.28444600000005,116.28344600000004,None,116.28444600000005,116.28444600000005,116.28544600000005,116.28544600000005,116.28444600000005,None,116.28544600000005,116.28544600000005,116.28644600000005,116.28644600000005,116.28544600000005,None,116.28644600000005,116.28644600000005,116.28744600000006,116.28744600000006,116.28644600000005,None,116.28744600000006,116.28744600000006,116.28844600000006,116.28844600000006,116.28744600000006,None,116.28844600000006,116.28844600000006,116.28944600000007,116.28944600000007,116.28844600000006,None,116.28944600000007,116.28944600000007,116.29044600000007,116.29044600000007,116.28944600000007,None,116.29044600000007,116.29044600000007,116.29144600000008,116.29144600000008,116.29044600000007,None,116.29144600000008,116.29144600000008,116.29244600000008,116.29244600000008,116.29144600000008,None,116.29244600000008,116.29244600000008,116.29344600000009,116.29344600000009,116.29244600000008,None,116.29344600000009,116.29344600000009,116.2944460000001,116.2944460000001,116.29344600000009,None,116.2944460000001,116.2944460000001,116.2954460000001,116.2954460000001,116.2944460000001,None,116.2954460000001,116.2954460000001,116.2964460000001,116.2964460000001,116.2954460000001,None,116.2964460000001,116.2964460000001,116.29744600000011,116.29744600000011,116.2964460000001,None,116.29744600000011,116.29744600000011,116.29844600000011,116.29844600000011,116.29744600000011,None,116.29844600000011,116.29844600000011,116.29944600000012,116.29944600000012,116.29844600000011,None,116.29944600000012,116.29944600000012,116.30044600000012,116.30044600000012,116.29944600000012,None,116.30044600000012,116.30044600000012,116.30144600000013,116.30144600000013,116.30044600000012,None,116.30144600000013,116.30144600000013,116.30244600000013,116.30244600000013,116.30144600000013,None,116.30244600000013,116.30244600000013,116.30344600000014,116.30344600000014,116.30244600000013,None,116.30344600000014,116.30344600000014,116.30444600000014,116.30444600000014,116.30344600000014,None,116.30444600000014,116.30444600000014,116.30544600000015,116.30544600000015,116.30444600000014,None,116.30544600000015,116.30544600000015,116.30644600000015,116.30644600000015,116.30544600000015,None,116.30644600000015,116.30644600000015,116.30744600000016,116.30744600000016,116.30644600000015,None,116.30744600000016,116.30744600000016,116.30844600000016,116.30844600000016,116.30744600000016,None,116.30844600000016,116.30844600000016,116.30944600000016,116.30944600000016,116.30844600000016,None,116.30944600000016,116.30944600000016,116.31044600000017,116.31044600000017,116.30944600000016,None,116.31044600000017,116.31044600000017,116.31144600000017,116.31144600000017,116.31044600000017,None,116.31144600000017,116.31144600000017,116.31244600000018,116.31244600000018,116.31144600000017,None,116.31244600000018,116.31244600000018,116.31344600000018,116.31344600000018,116.31244600000018,None,116.31344600000018,116.31344600000018,116.31444600000019,116.31444600000019,116.31344600000018,None,116.31444600000019,116.31444600000019,116.3154460000002,116.3154460000002,116.31444600000019,None,116.3154460000002,116.3154460000002,116.3164460000002,116.3164460000002,116.3154460000002,None,116.3164460000002,116.3164460000002,116.3174460000002,116.3174460000002,116.3164460000002,None,116.3174460000002,116.3174460000002,116.31844600000021,116.31844600000021,116.3174460000002,None,116.31844600000021,116.31844600000021,116.31944600000021,116.31944600000021,116.31844600000021,None,116.31944600000021,116.31944600000021,116.32044600000022,116.32044600000022,116.31944600000021,None,116.32044600000022,116.32044600000022,116.32144600000022,116.32144600000022,116.32044600000022,None,116.32144600000022,116.32144600000022,116.32244600000023,116.32244600000023,116.32144600000022,None,116.32244600000023,116.32244600000023,116.32344600000023,116.32344600000023,116.32244600000023,None,116.32344600000023,116.32344600000023,116.32444600000024,116.32444600000024,116.32344600000023,None,116.32444600000024,116.32444600000024,116.32544600000024,116.32544600000024,116.32444600000024,None,116.32544600000024,116.32544600000024,116.32644600000025,116.32644600000025,116.32544600000024,None,116.32644600000025,116.32644600000025,116.32744600000025,116.32744600000025,116.32644600000025,None,116.32744600000025,116.32744600000025,116.32844600000026,116.32844600000026,116.32744600000025,None,116.32844600000026,116.32844600000026,116.32944600000026,116.32944600000026,116.32844600000026,None,116.32944600000026,116.32944600000026,116.33044600000026,116.33044600000026,116.32944600000026,None,116.33044600000026,116.33044600000026,116.33144600000027,116.33144600000027,116.33044600000026,None,116.33144600000027,116.33144600000027,116.33244600000027,116.33244600000027,116.33144600000027,None,116.33244600000027,116.33244600000027,116.33344600000028,116.33344600000028,116.33244600000027,None,116.33344600000028,116.33344600000028,116.33444600000028,116.33444600000028,116.33344600000028,None,116.33444600000028,116.33444600000028,116.33544600000029,116.33544600000029,116.33444600000028,None,116.33544600000029,116.33544600000029,116.3364460000003,116.3364460000003,116.33544600000029,None,116.3364460000003,116.3364460000003,116.3374460000003,116.3374460000003,116.3364460000003,None,116.3374460000003,116.3374460000003,116.3384460000003,116.3384460000003,116.3374460000003,None,116.3384460000003,116.3384460000003,116.33944600000031,116.33944600000031,116.3384460000003,None,116.33944600000031,116.33944600000031,116.34044600000031,116.34044600000031,116.33944600000031,None,116.34044600000031,116.34044600000031,116.34144600000032,116.34144600000032,116.34044600000031,None,116.34144600000032,116.34144600000032,116.34244600000032,116.34244600000032,116.34144600000032,None,116.34244600000032,116.34244600000032,116.34344600000033,116.34344600000033,116.34244600000032,None,116.34344600000033,116.34344600000033,116.34444600000033,116.34444600000033,116.34344600000033,None,116.34444600000033,116.34444600000033,116.34544600000034,116.34544600000034,116.34444600000033,None,116.34544600000034,116.34544600000034,116.34644600000034,116.34644600000034,116.34544600000034,None,116.34644600000034,116.34644600000034,116.34744600000035,116.34744600000035,116.34644600000034,None,116.34744600000035,116.34744600000035,116.34844600000035,116.34844600000035,116.34744600000035,None,116.34844600000035,116.34844600000035,116.34944600000036,116.34944600000036,116.34844600000035,None,116.34944600000036,116.34944600000036,116.35044600000036,116.35044600000036,116.34944600000036,None,116.35044600000036,116.35044600000036,116.35144600000037,116.35144600000037,116.35044600000036,None,116.35144600000037,116.35144600000037,116.35244600000037,116.35244600000037,116.35144600000037,None,116.35244600000037,116.35244600000037,116.35344600000037,116.35344600000037,116.35244600000037,None,116.35344600000037,116.35344600000037,116.35444600000038,116.35444600000038,116.35344600000037,None,116.35444600000038,116.35444600000038,116.35544600000038,116.35544600000038,116.35444600000038,None,116.35544600000038,116.35544600000038,116.35644600000039,116.35644600000039,116.35544600000038,None,116.35644600000039,116.35644600000039,116.3574460000004,116.3574460000004,116.35644600000039,None,116.3574460000004,116.3574460000004,116.3584460000004,116.3584460000004,116.3574460000004,None,116.3584460000004,116.3584460000004,116.3594460000004,116.3594460000004,116.3584460000004,None,116.3594460000004,116.3594460000004,116.36044600000041,116.36044600000041,116.3594460000004,None,116.36044600000041,116.36044600000041,116.36144600000041,116.36144600000041,116.36044600000041,None,116.36144600000041,116.36144600000041,116.36244600000042,116.36244600000042,116.36144600000041,None,116.36244600000042,116.36244600000042,116.36344600000042,116.36344600000042,116.36244600000042,None,116.36344600000042,116.36344600000042,116.36444600000043,116.36444600000043,116.36344600000042,None,116.36444600000043,116.36444600000043,116.36544600000043,116.36544600000043,116.36444600000043,None,116.36544600000043,116.36544600000043,116.36644600000044,116.36644600000044,116.36544600000043,None,116.36644600000044,116.36644600000044,116.36744600000044,116.36744600000044,116.36644600000044,None,116.36744600000044,116.36744600000044,116.36844600000045,116.36844600000045,116.36744600000044,None,116.36844600000045,116.36844600000045,116.36944600000045,116.36944600000045,116.36844600000045,None,116.36944600000045,116.36944600000045,116.37044600000046,116.37044600000046,116.36944600000045,None,116.37044600000046,116.37044600000046,116.37144600000046,116.37144600000046,116.37044600000046,None,116.37144600000046,116.37144600000046,116.37244600000047,116.37244600000047,116.37144600000046,None,116.37244600000047,116.37244600000047,116.37344600000047,116.37344600000047,116.37244600000047,None,116.37344600000047,116.37344600000047,116.37444600000048,116.37444600000048,116.37344600000047
          ],

    #lat =[dflatiude['A'],dflatiude['B'],dflatiude['C'],dflatiude['D'],dflatiude['E'],dflatiude['F']])
    lat = [
           39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,39.985702,39.986702,39.986702,39.985702,39.985702,None,
           39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,39.986702,39.987702,39.987702,39.986702,39.986702,None,
           39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,39.987702,39.988702,39.988702,39.987702,39.987702,None,
           39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,39.988702,39.989702,39.989702,39.988702,39.988702,None,
           39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,39.989702,39.990702,39.990702,39.989702,39.989702,None,
           39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,39.990702,39.991702,39.991702,39.990702,39.990702,None,
           39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,39.991702,39.992702,39.992702,39.991702,39.991702,None,
           39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,39.992702,39.993702,39.993702,39.992702,39.992702,None,
           39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,39.993702,39.994702,39.994702,39.993702,39.993702,None,
           39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,39.994702,39.995702,39.995702,39.994702,39.994702,None,
           39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,39.995702,39.996702,39.996702,39.995702,39.995702,None,
           39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,39.996702,39.997702,39.997702,39.996702,39.996702,None,
           39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,39.997702,39.998702,39.998702,39.997702,39.997702,None,
           39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,39.998702,39.999702,39.999702,39.998702,39.998702,None,
           39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,39.999702,40.000702,40.000702,39.999702,39.999702,None,
           40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,40.000702,40.001702,40.001702,40.000702,40.000702,None,
           40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,40.001702,40.002702,40.002702,40.001702,40.001702,None,
           40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,40.002702,40.003702,40.003702,40.002702,40.002702,None,
           40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,40.003702,40.004702,40.004702,40.003702,40.003702,None,
           40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,40.004702,40.005702,40.005702,40.004702,40.004702,None,
           40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,40.005702,40.006702,40.006702,40.005702,40.005702,None,
           40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,40.006702,40.007702,40.007702,40.006702,40.006702,None,
           40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,40.007702,40.008702,40.008702,40.007702,40.007702,None,
           40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702,None,40.008702,40.009702,40.009702,40.008702,40.008702

          ])
    )

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=rs['lat'][0],
            lon=rs['lgt'][0]
        ),
        pitch=0,
        zoom=13,
        style='light'
    ),
)
fig.show()

time: 9.05 s


In [ ]:
# plot the final reduced set of coordinate points vs the original full set
fig, ax = plt.subplots(figsize=[10, 6])
rs_scatter = ax.scatter(rs['lgt'], rs['lat'], c='#99cc99', edgecolor='None', alpha=0.7, s=120)
df_scatter = ax.scatter(df['lgt'], df['lat'], c='k', alpha=0.9, s=3)
ax.set_title('Full data set vs DBSCAN reduced set')
ax.set_xlabel('Latitude')
ax.set_ylabel('Longitude')
ax.legend([df_scatter, rs_scatter], ['Full set', 'Reduced set'], loc='upper right')
plt.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scattermapbox(
    fill = "toself",
    lon = [-74, -70, -70, -74,-74], lat = [47, 47, 45, 45,45],
    marker = { 'size': 10, 'color': "orange" }))

fig.update_layout(
    mapbox = {
        'style': "stamen-terrain",
        'center': {'lon': -73, 'lat': 46 },
        'zoom': 5},
    showlegend = False)

fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scattermapbox(
    mode = "lines", fill = "toself",
    lon = [116.318417, 116.318417, 116.319417,116.319427, 116.318417, None, 116.319427, 116.319427, 116.320527, 116.320527, 116.319427, None, 116.320527, 116.320527, 116.311527,116.311527,116.320527, None, 100, 100, 80, 80, 100],
    lat = [39.984702, 39.985702, 39.985710, 39.984702, 39.984702, None, 39.984702, 39.985702,39.985702, 39.984702, 39.984702, None, 39.985702, 39.986702, 39.986702, 39.985702, 39.985702, None, 40, 50, 50, 40, 40]))

fig.update_layout(
    mapbox = {'style': "stamen-terrain", 'center': {'lon': 70, 'lat': 42}, 'zoom': 2},
    showlegend = False,
    margin = {'l':0, 'r':0, 'b':0, 't':0})

fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scattermapbox(
    mode = "markers",
    lon = [-73.605], lat = [45.51],
    marker = {'size': 20, 'color': ["cyan"]}))

fig.update_layout(
    mapbox = {
        'style': "stamen-terrain",
        'center': { 'lon': -73.6, 'lat': 45.5},
        'zoom': 12, 'layers': [{
            'source': {
                'type': "FeatureCollection",
                'features': [{
                    'type': "Feature",
                    'geometry': {
                        'type': "MultiPolygon",
                        'coordinates': [[[
                            [-73.606352888, 45.507489991], [-73.606133883, 45.50687600],
                            [-73.605905904, 45.506773980], [-73.603533905, 45.505698946],
                            [-73.602475870, 45.506856969], [-73.600031904, 45.505696003],
                            [-73.599379992, 45.505389066], [-73.599119902, 45.505632008],
                            [-73.598896977, 45.505514039], [-73.598783894, 45.505617001],
                            [-73.591308727, 45.516246185], [-73.591380782, 45.516280145],
                            [-73.596778656, 45.518690062], [-73.602796770, 45.521348046],
                            [-73.612239983, 45.525564037], [-73.612422919, 45.525642061],
                            [-73.617229085, 45.527751983], [-73.617279234, 45.527774160],
                            [-73.617304713, 45.527741334], [-73.617492052, 45.527498362],
                            [-73.617533258, 45.527512253], [-73.618074188, 45.526759105],
                            [-73.618271651, 45.526500673], [-73.618446320, 45.526287943],
                            [-73.618968507, 45.525698560], [-73.619388002, 45.525216750],
                            [-73.619532966, 45.525064183], [-73.619686662, 45.524889290],
                            [-73.619787038, 45.524770086], [-73.619925742, 45.524584939],
                            [-73.619954486, 45.524557690], [-73.620122362, 45.524377961],
                            [-73.620201713, 45.524298907], [-73.620775593, 45.523650879]
                        ]]]
                    }
                }]
            },
            'type': "fill", 'below': "traces", 'color': "royalblue"}]},
    margin = {'l':0, 'r':0, 'b':0, 't':0})

fig.show()